# Python Marcação da sílaba tónica

Passo a passo da elaboração do ficheiro python e da aplicação das expressões regulares da marcação da sílaba tónica ao corpus 2.

Se necessário, instalar pacotes `pandas`, `regex` e `odfpy`.

In [1]:
#!pip install pandas
#!pip install regex
#!pip install odfpy

Importar as bibliotecas necessárias para o processamento: `pandas` para a manipulação de dados, `regex` para as expressões regulares com lookbehind variáveis, e `IPython.display` para a visualização dos resultados no notebook.

In [2]:
import pandas as pd
import regex as re  # Use the regex module for variable-width look-behinds
from IPython.display import display

Definir a função `load_column_to_list`, que lê a coluna A do ficheiro ODS e devolve a lista de palavras.

In [3]:
def load_column_to_list(file_path, sheet_name):
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
        column_data = df[0].dropna().tolist()
        return column_data
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
    except ValueError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Definir a função `load_regex_to_dict`, que lê as expressões regulares e os respetivos IDs e substituições a partir de uma folha ODS.

In [4]:
def load_regex_to_dict(file_path, sheet_name):
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
        regex_dict = {str(k): {"replacement": str(v), "id": str(i)} for k, v, i in zip(df[0], df[1], df[2])}
        return regex_dict
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
    except ValueError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Definir a função `apply_patterns_to_words` que aplica todas as expressões regulares às palavras da lista. Gera um ficheiro de log com o histórico de transformações e regista os IDs das regras aplicadas.

In [5]:
def apply_patterns_to_words(words_list, regex_dict, log_file_path="../Debug/debug_acento_notebook.txt"):
    results = []
    with open(log_file_path, "w", encoding="utf-8") as log_file:
        for word in words_list:
            processed_word = word
            used_ids = []
            for pattern, data in regex_dict.items():
                replacement = data["replacement"]
                rule_id = data["id"]
                try:
                    if replacement == "nan":
                        replacement = ""
                    new_word = re.sub(pattern, replacement, processed_word)
                    if new_word != processed_word:
                        used_ids.append(rule_id)
                        log_file.write(f"Original Word: {processed_word} -> Processed Word: {new_word}\n")
                        log_file.write(f"Pattern Applied: {pattern} -> {replacement} (Pattern id: {rule_id})\n\n")
                    processed_word = new_word
                except re.error as e:
                    print(f"Regex error for pattern '{pattern}': {e}")
            results.append((processed_word, used_ids))
    return results

Definir os caminhos para o ficheiro ODS e os nomes das folhas que contêm a lista de palavras e as expressões regulares.

In [6]:
file_path = "../ODS/Acento.ods"
folha_palavras = "Palavras"
folha_expressoes = "ExpressoesAcento"

Carregar a lista de palavras e o dicionário de expressões regulares a partir do ficheiro ODS.

In [7]:
words_list = load_column_to_list(file_path, folha_palavras)
regex_dict = load_regex_to_dict(file_path, folha_expressoes)

Mostrar a lista de palavras e o dicionário de expressões carregados.

In [8]:
print(words_list)
print(regex_dict)

['alto', 'amarelo', 'amigo', 'antigo', 'assado', 'azul', 'baixo', 'belo', 'bom', 'bonito', 'branco', 'capaz', 'caro', 'castanho', 'cheio', 'chinês', 'claro', 'comprido', 'contente', 'diferente', 'difícil', 'doce', 'enorme', 'escuro', 'estranho', 'fácil', 'famoso', 'feliz', 'fino', 'forte', 'fresco', 'frio', 'grande', 'igual', 'importante', 'junto', 'leve', 'limpo', 'lindo', 'livre', 'longo', 'mau', 'natural', 'negro', 'normal', 'novo', 'obrigado', 'ótimo', 'passado', 'pequeno', 'pobre', 'português', 'possível', 'preciso', 'preto', 'principal', 'pronto', 'quente', 'rico', 'seco', 'simpático', 'simples', 'sozinho', 'triste', 'último', 'velho', 'vermelho', 'vivo', 'agora', 'aí', 'ainda', 'ali', 'amanhã', 'apenas', 'aqui', 'assim', 'até', 'bastante', 'bem', 'cá', 'cedo', 'demasiado', 'depois', 'depressa', 'embora', 'então', 'entretanto', 'finalmente', 'geralmente', 'hoje', 'imenso', 'já', 'lá', 'logo', 'longe', 'mais', 'mal', 'menos', 'mesmo', 'muito', 'nada', 'não', 'nem', 'normalmente', 

Aplicar as expressões regulares às palavras, registando os resultados processados e os IDs das regras usadas.

In [9]:
if words_list and regex_dict:
    processed_data = apply_patterns_to_words(words_list, regex_dict)
    
    results_for_df = []
    for original, (processed, ids_used) in zip(words_list, processed_data):
        results_for_df.append({
            "Original": original,
            "Processado": processed,
            "IDs usados": ", ".join(ids_used) if ids_used else ""
        })
    
    df_resultados = pd.DataFrame(results_for_df)

## Resultados

Apresentar os resultados da transcrição fonética: palavra original, palavra processada e IDs das expressões regulares aplicadas.

In [10]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 0):
    display(df_resultados)

,Original,Processado,IDs usados
0,alto,ˈalto,A44
1,amarelo,amaˈrelo,A25
2,amigo,aˈmigo,A24
3,antigo,anˈtigo,A24
4,assado,aˈssado,A24
5,azul,aˈzul,A7
6,baixo,ˈbaixo,A24
7,belo,ˈbelo,A24
8,bom,ˈbom,A20
9,bonito,boˈnito,A24
